In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install comet_ml

In [ ]:
# import comet_ml at the top of your file
from comet_ml import Experiment
import comet_ml
import logging
# Create an experiment with your api key
experiment = Experiment(
    project_name="digit-recognizer",
    workspace="wodenwang820118",
    api_key="Your API key")
)

In [ ]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger("comet_ml")

In [ ]:
# tensorflow 2.7 
from tensorflow import keras
from tensorflow.keras.layers import Input,Flatten,Dense,Dropout,BatchNormalization,Conv2D,MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
from tensorflow.keras import activations

In [ ]:
import pandas as pd
from keras.utils.np_utils import to_categorical
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras

%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
df_train=pd.read_csv('/content/drive/MyDrive/Mnist_CNN/train.csv')
# df_train.head(1)

In [ ]:
df_train.shape

In [ ]:
X = df_train.drop('label', axis=1)
y = df_train['label']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# give the input shape one more dimension to fit the CNN
X_train = X_train.values.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.values.reshape(X_test.shape[0], 28, 28, 1)
y_train = y_train.values.reshape(y_train.shape[0], 1)
y_test = y_test.values.reshape(y_test.shape[0], 1)

In [ ]:
# make sure the division gets the float
X_train = X_train.astype('float32')
X_test =  X_test.astype('float32')

# normalize the data in to RGB range
X_train /= 255
X_test /= 255

In [ ]:
print(X_train.shape)
print("Max value in X_train:", X_train.max())
print("Min value in X_train:", X_train.min())

In [ ]:
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [ ]:
print("Shape of y_train:", y_train.shape)
print("One value of y_train:", y_train[2])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=3, verbose=1, mode='auto', min_delta=0.00001, cooldown=0, min_lr=0.0001)

In [ ]:
class DigitClassifier:
    def __init__(self,X_train,X_test,y_train,y_test,early_stop,reduce_lr,experiment):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.reduce_lr = reduce_lr
        self.early_stop = early_stop
        self.experiment = experiment
    
    def build_model(self):
        # Build the model
        model = Sequential()
        # padding='same' -> fill in the empty padding while convolve
        # kernel_size=(3,3) -> the image pixels (28,28) will extract (3,3) at a time and multiply the random numbers of filter at the same size
        # filters reference: https://www.youtube.com/watch?v=cNBBNAxC8l4&ab_channel=deeplizard
        # reference: https://www.youtube.com/watch?v=YRhxdVk_sIs&ab_channel=deeplizard
        model.add(
            Conv2D(
              filters=self.experiment.get_parameter("first_layer_filters"),
              kernel_size=(3,3),
              input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]),
              padding='same'
          )
        )
        # https://www.youtube.com/watch?v=ZjM_XQa5s6s&ab_channel=deeplizard
        model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

        model.add(
            Conv2D(
              filters=self.experiment.get_parameter("second_layer_filters"),
              kernel_size=(4,4),
              padding='same'
          )
        )
        model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))
        model.add(Flatten())

        model.add(
            Dense(
                units=self.experiment.get_parameter("second_layer_units"),
                )
            )
        model.add(layers.Activation(activations.elu))
        model.add(Dropout(self.experiment.get_parameter("second_layer_dropout_rate")))
        
        model.add(Dense(10,activation='softmax'))
        optimizer = keras.optimizers.Adam(0.001)
        model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model
    
    def train_model(self):
        # Train the model
        model = self.build_model()
        model.fit(
            self.X_train,
            self.y_train,
            batch_size=self.experiment.get_parameter("batch_size"),
            epochs=self.experiment.get_parameter("epochs"),
            validation_data=(self.X_test,self.y_test),
            callbacks=[self.early_stop,self.reduce_lr]
        )
        return model
    
    def evaluate_model(self):
        # Evaluate the model
        model = self.train_model()
        score = model.evaluate(self.X_test,self.y_test)
        LOGGER.info(f"{ score }")
    
    def grid_search(self, config_dict):
        opt = comet_ml.Optimizer(config_dict, api_key="Your API key")
        for self.experiment in opt.get_experiments(project_name="digit-recognizer"):

            self.build_model()
            self.train_model()
            self.evaluate_model()
            self.experiment.end()

In [ ]:
classifier = DigitClassifier(X_train,X_test,y_train,y_test,early_stop,reduce_lr,experiment)
# mu is the mean number of units, sigma is the standard deviation
classifier.grid_search({
    "algorithm": "bayes",
    "name": "Optimize Music Classification Network",
    "spec": {"maxCombo": 10, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "first_layer_filters":{"type": "discrete", "values": [16,32,48,64]},
        "second_layer_filters":{"type": "discrete", "values": [16,32,48,64]},
        "second_layer_units": {"type": "discrete", "values": [300,400,450,500,550,600,650,700,750,800]},
        "second_layer_dropout_rate": {"type": "discrete", "values": [0.1,0.2,0.3,0.4,0.5,0.6]},
        "batch_size": {"type": "discrete", "values": [32,64]},
        "epochs": {"type": "discrete", "values": [150]},
    },
    "name": "digit-recognizer",
    "trials": 1,
})

In [ ]:
# Build the model
model = Sequential()
model.add(
    Conv2D(
      filters=64,
      kernel_size=(3,3),
      input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]),
      padding='same'
  )
)
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))
# model.add(layers.Activation(activations.elu))

model.add(
    Conv2D(
      filters=32,
      kernel_size=(4,4),
      padding='same'
  )
)
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))
# model.add(layers.Activation(activations.elu))
model.add(Flatten())

model.add(
    Dense(
        units=650,
        )
    )
model.add(layers.Activation(activations.elu))
model.add(Dropout(0.4))

model.add(Dense(10,activation='softmax'))
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(X_train,y_train,batch_size=64,epochs=200,validation_data=(X_test,y_test),callbacks=[early_stop,reduce_lr])

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/Mnist_CNN/test.csv')
test_data = df_test.values.reshape(df_test.shape[0], 28, 28, 1)

In [ ]:
# make sure the division gets the float
test_data = test_data.astype('float32')

# normalize the data in to RGB range
test_data /= 255

In [ ]:
import numpy as np

In [ ]:
predictions = model.predict(test_data)
labels = np.argmax(predictions, axis=1)

In [ ]:
df_result = pd.DataFrame(np.arange(1,len(labels)+1), columns=['ImageId'])
df_result['label']=labels
df_result.to_csv('/content/drive/MyDrive/Mnist_CNN/submission_cnn_comet_keras.csv', index=False)